# On the pertinence of LLMs for ontology learning: results analysis

This notebook contains the functions we used to construct our results. 
We try to automate as much as possible our analysis, particularly leveraging some SPARQL queries.
However, human are still needed (Yes!) to align the ontologies constructed with OLAF with our base one, i.e., the pizza ontology.
In particular, we needed to align the ontologies independently of the exact labels.
Hence, to help us in this endeavour, we used a basic Hamming distance on the labels or end of the URI if no label were available.
In the end the counting was done manually though. 

## Notebook setup

In [1]:
import os

from rapidfuzz import process, fuzz
from rdflib import Graph, BNode
from rdflib.namespace import Namespace

import pandas as pd

In [2]:
# Define some namespaces to ease the URIs manipulation and visualisation
olaf_eswc_ns = Namespace("https://github.com/wikit-ai/olaf-llm-eswc2024/o/example#")
pizza_onto_ns = Namespace("http://www.co-ode.org/ontologies/pizza/pizza.owl#")

olaf_eswc_ns_bindings = {
    "olaf-eswc": olaf_eswc_ns,
    "pizza-onto": pizza_onto_ns
}

In [3]:
# Load the different RDF graphs
pizza_onto_graph = Graph()
pizza_onto_graph.parse(os.path.join(os.getenv("DATA_PATH"), "pizza_onto_ground_truth.ttl"))

llm_text2owl_graph = Graph()
llm_text2owl_graph.parse(os.path.join(os.getenv("RESULTS_PATH"), "llm_text_to_owl", "llm_owl_pizza_onto_eswc2024.ttl"))

olaf_llm_graph = Graph()
olaf_llm_graph.parse(os.path.join(os.getenv("RESULTS_PATH"), "llm_pipeline", "llm_pipeline_kr_rdf_graph_eswc2024.ttl"))

olaf_no_llm_graph = Graph()
olaf_no_llm_graph.parse(os.path.join(os.getenv("RESULTS_PATH"), "no_llm_pipeline", "no_llm_pipeline_kr_rdf_graph_eswc2024.ttl"))

<Graph identifier=Nf84d304737a049fa9e10406efce16766 (<class 'rdflib.graph.Graph'>)>

## SPARQL tools

In [4]:
# Some SPARQL queries we use for our analysis
owl_classes_sparql_q = """
            SELECT DISTINCT ?class WHERE {
                ?class rdf:type owl:Class .
            }
        """

owl_classes_labels_sparql_q = """
            SELECT ?label WHERE {
                ?class rdf:type owl:Class ;
                        rdfs:label ?label .
            }
        """

owl_classes_en_labels_sparql_q = """
            SELECT ?label WHERE {
                ?class rdf:type owl:Class ;
                        rdfs:label ?label .
                FILTER(LANG(?label) = "en").
            }
        """

owl_obj_props_sparql_q = """
            SELECT DISTINCT ?prop WHERE {
                ?prop rdf:type owl:ObjectProperty .
            }
        """

owl_named_individuals_sparql_q = """
            SELECT DISTINCT ?ind WHERE {
                ?ind rdf:type owl:NamedIndividual .
            }
        """

owl_named_ind_from_obj_props_sparql_q = """
            SELECT DISTINCT ?ind WHERE {
                {
                    ?prop rdf:type owl:ObjectProperty .
                    ?ind ?prop ?o .
                }
                UNION
                {
                    ?prop rdf:type owl:ObjectProperty .
                    ?s ?prop ?ind .
                }
                UNION
                {
                    ?ind rdf:type ?class .
                    ?class rdf:type owl:Class .
                }
                UNION
                {
                    ?ind rdf:type owl:NamedIndividual .
                }
            }
        """

owl_named_classes_ind_from_obj_props_sparql_q = """
            SELECT DISTINCT ?ind WHERE {
                {
                    ?prop rdf:type owl:ObjectProperty .
                    ?ind ?prop ?o .
                }
                UNION
                {
                    ?prop rdf:type owl:ObjectProperty .
                    ?s ?prop ?ind .
                }
                UNION
                {
                    ?ind rdf:type ?class .
                    ?class rdf:type owl:Class .
                }
                UNION
                {
                    ?ind rdf:type owl:NamedIndividual .
                }
                UNION
                {
                    ?ind rdf:type owl:Class .
                }
            }
        """

rdfs_subclassof_tuples_sparql_q = """
            SELECT DISTINCT ?child ?parent WHERE {
                ?child rdfs:subClassOf ?parent .
            }
        """

In [5]:
def get_sparql_q_res_fragments_skip_bnodes(sparql_q: str, graph: Graph, ns: dict[str, Namespace]) -> set[tuple]:
    """Run a SPARQL query over an RDF graph and return the URIs fragments skipping blank nodes.

    Parameters
    ----------
    sparql_q : str
        The SPARQL query.
    graph : Graph
        The RDF graph.
    ns : dict[str, Namespace]
        Namespaces to use to extract URIs fragments (i.e., end).

    Returns
    -------
    set[tuple]
        The results.
    """
    q_res = graph.query(sparql_q, initNs=ns)
    
    fragments = set()
    for res in q_res:
        t = []
        for item in res:
            if not isinstance(item, BNode):
                t.append(item.fragment)
        fragments.add(tuple(t))

    return fragments

In [6]:
def get_sparql_q_label_res(sparql_q: str, graph: Graph, ns: dict[str, Namespace]) -> set[tuple]:
    """Run a SPARQL query over an RDF graph and return the URIs labels.

    The SPARQL query must return labels, i.e., strings.

    Parameters
    ----------
    sparql_q : str
        The SPARQL query.
    graph : Graph
        The RDF graph.
    ns : dict[str, Namespace]
        Namespaces to use to extract URIs fragments (i.e., end).

    Returns
    -------
    set[tuple]
        The results.
    """
    q_res = graph.query(sparql_q, initNs=ns)

    labels = {str(res[0]) for res in q_res}

    return labels

In [7]:
def get_sparql_q_tuple_res(sparql_q: str, graph: Graph, ns: dict[str, Namespace]) -> set[tuple]:
    """Run a SPARQL query over an RDF graph and return the URIs labels.

    The SPARQL query must return labels, i.e., strings.

    Parameters
    ----------
    sparql_q : str
        The SPARQL query.
    graph : Graph
        The RDF graph.
    ns : dict[str, Namespace]
        Namespaces to use to extract URIs fragments (i.e., end).

    Returns
    -------
    set[tuple]
        The results.
    """
    q_res = graph.query(sparql_q, initNs=ns)

    labels = {(str(res[0]), str(res[1])) for res in q_res}

    return labels

## Ontologies overview

In [8]:
# Extract ontology components
llm_text2owl_g_classes = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_classes_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_classes = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_classes_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_classes = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_classes_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_classes = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_classes_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

llm_text2owl_g_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

llm_text2owl_g_individuals = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_individuals_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_individuals = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_individuals_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_individuals = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_individuals_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_individuals = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_individuals_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

llm_text2owl_g_individuals_from_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_individuals_from_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_individuals_from_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_individuals_from_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

llm_text2owl_g_subclassof_t = get_sparql_q_res_fragments_skip_bnodes(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_subclassof_t = get_sparql_q_res_fragments_skip_bnodes(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_subclassof_t = get_sparql_q_res_fragments_skip_bnodes(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_subclassof_t = get_sparql_q_res_fragments_skip_bnodes(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

In [9]:
# construct the counts table for visualisation
onto_overview = {
    "Count": [
        "OWL named classes", 
        "OWL object properties", 
        "OWL named individuals", 
        "OWL named individuals from OWL object properties", 
        "RDFS subClassOf tuples"
    ],
    "Pizza": [
        len(pizza_onto_g_classes), 
        len(pizza_onto_g_props), 
        len(pizza_onto_g_individuals), 
        len(pizza_onto_g_individuals_from_props), 
        len(pizza_onto_g_subclassof_t)
    ],
    "Text to OWL": [
        len(llm_text2owl_g_classes), 
        len(llm_text2owl_g_props), 
        len(llm_text2owl_g_individuals), 
        len(llm_text2owl_g_individuals_from_props), 
        len(llm_text2owl_g_subclassof_t)
    ],
    "OLAF LLM": [
        len(olaf_llm_g_classes), 
        len(olaf_llm_g_props), 
        len(olaf_llm_g_individuals), 
        len(olaf_llm_g_individuals_from_props), 
        len(olaf_llm_g_subclassof_t)
    ],
    "OLAF no LLM": [
        len(olaf_no_llm_g_classes), 
        len(olaf_no_llm_g_props), 
        len(olaf_no_llm_g_individuals), 
        len(olaf_no_llm_g_individuals_from_props), 
        len(olaf_no_llm_g_subclassof_t)
    ]
}

df_overview = pd.DataFrame(onto_overview)

In [10]:
df_overview

,Count,Pizza,Text to OWL,OLAF LLM,OLAF no LLM
0,OWL named classes,97,36,99,111
1,OWL object properties,8,2,77,22
2,OWL named individuals,5,0,0,343
3,OWL named individuals from OWL object properties,5,27,97,343
4,RDFS subClassOf tuples,141,33,114,390


Analyse juste en regardant les chiffres, ordre de grandeurs : 

- Text to OWl
  - c'est cool mais assez limité, très simplifié
- OLAF LLM
  - c'est le top, c'est ce qui colle le mieux
- OLAF no LLM
  - looks but the number suggest a big mess

Look at class number and combination of class and individuals

## Ontologies evaluation

Classes

In [11]:
pizza_onto_class_labels = get_sparql_q_label_res(sparql_q=owl_classes_en_labels_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_class_labels = {label.lower().replace(" ", "") for label in pizza_onto_class_labels}

In [12]:
llm_text2owl_class_labels = get_sparql_q_label_res(sparql_q=owl_classes_labels_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_class_labels = list({label.lower().replace(" ", "") for label in llm_text2owl_class_labels})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_class_labels:
#     res = process.extract(true_label, llm_text2owl_class_labels, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [13]:
common_classes_pizza_text2owl=36

In [14]:
olaf_llm_class_labels = get_sparql_q_label_res(sparql_q=owl_classes_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_class_labels = list({label.lower().replace("https://github.com/wikit-ai/olaf-llm-eswc2024/o/example#", "") for label in olaf_llm_class_labels})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_class_labels:
#     res = process.extract(true_label, olaf_llm_class_labels, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [15]:
common_classes_pizza_olaf_llm=55

In [16]:
olaf_no_llm_class_labels = get_sparql_q_label_res(sparql_q=owl_classes_labels_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_class_labels = list({label.lower().replace(" ", "") for label in olaf_no_llm_class_labels})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_class_labels:
#     res = process.extract(true_label, olaf_no_llm_class_labels, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [17]:
common_classes_pizza_olaf_no_llm=43

Individuals

In [18]:
pizza_onto_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_ind_uri = list({label.lower().replace(str(pizza_onto_ns), "") for label in pizza_onto_ind_uri})

In [19]:
llm_text2owl_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in llm_text2owl_ind_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_ind_uri:
#     res = process.extract(true_label, llm_text2owl_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [20]:
common_ind_pizza_text2owl=0

In [21]:
olaf_llm_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_llm_ind_uri})
for true_label in pizza_onto_ind_uri:
    res = process.extract(true_label, olaf_llm_ind_uri, scorer=fuzz.WRatio, limit=3)
    print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 

england :  leavenedwheatbaseddough (51.42857142857142), thinandcrispybase (51.42857142857142), vegetarian (47.05882352941176)
italy :  italianorigin (80.0), widevariety (60.00000000000001), cheesyvegetabletopping (54.0)
germany :  vegetarian (58.82352941176471), parmesan (53.333333333333336), nonvegetarianpizza (51.42857142857142)
america :  americanhotpizza (90.0), ham (72.0), artichoketopping (60.00000000000001)
france :  vegetarian (54.0), prawnstopping (49.090909090909086), fourcheesestopping (49.090909090909086)


In [22]:
common_ind_pizza_olaf_llm = 1

In [23]:
olaf_no_llm_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_no_llm_ind_uri})
for true_label in pizza_onto_ind_uri:
    res = process.extract(true_label, olaf_no_llm_ind_uri, scorer=fuzz.WRatio, limit=3)
    print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})")

england :  _ad (60.00000000000001), _cookingpan (60.00000000000001), _artichokeplant (55.38461538461539)
italy :  _italian (72.0), _italianorigin (72.0), _ordinarily (67.5)
germany :  _vegetarian (55.55555555555556), _giardiniera (54.0), _optionslikeprawns (51.42857142857142)
america :  _america (93.33333333333333), _americanhot (90.0), _americanhotpizza (90.0)
france :  _spinaciaoleracea (65.45454545454547), _broadness (60.00000000000001), _anchovies (54.0)


In [24]:
common_ind_pizza_olaf_no_llm = 2

Classes + indiviuals

In [25]:
pizza_onto_classes_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_classes_ind_from_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_classes_ind_uri = list({label.lower().replace(str(pizza_onto_ns), "") for label in pizza_onto_classes_ind_uri})
pizza_onto_classes_ind_uri = [uri for uri in pizza_onto_classes_ind_uri if not(uri[0:2]=="nc")]

In [26]:
llm_text2owl_classes_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_classes_ind_from_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_classes_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in llm_text2owl_classes_ind_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_classes_ind_uri:
#     res = process.extract(true_label, llm_text2owl_classes_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [27]:
common_classes_ind_pizza_text2owl=34

In [28]:
olaf_llm_classes_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_classes_ind_from_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_classes_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_llm_classes_ind_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_classes_ind_uri:
#     res = process.extract(true_label, olaf_llm_classes_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [29]:
common_classes_ind_pizza_olaf_llm=54

In [30]:
olaf_no_llm_classes_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_classes_ind_from_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_classes_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_no_llm_classes_ind_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_classes_ind_uri:
#     res = process.extract(true_label, olaf_no_llm_classes_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 

In [31]:
common_classes_ind_pizza_olaf_no_llm=59

Object properties

In [32]:
pizza_onto_obj_prop_uri = get_sparql_q_label_res(sparql_q=owl_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_obj_prop_uri = list({label.lower().replace(str(pizza_onto_ns), "") for label in pizza_onto_obj_prop_uri})

In [33]:
llm_text2owl_obj_prop_uri = get_sparql_q_label_res(sparql_q=owl_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_obj_prop_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in llm_text2owl_obj_prop_uri})

for true_label in pizza_onto_obj_prop_uri:
    res = process.extract(true_label, llm_text2owl_obj_prop_uri, scorer=fuzz.WRatio, limit=2)
    print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]})") 
    

hasbase :  hasbase (100.0), hastopping (35.29411764705882)
isbaseof :  hasbase (66.66666666666667), hastopping (33.333333333333336)
hascountryoforigin :  hasbase (54.0), hastopping (52.94117647058824)
isingredientof :  hastopping (33.333333333333336), hasbase (25.71428571428571)
istoppingof :  hastopping (76.19047619047619), hasbase (20.0)
hasspiciness :  hasbase (65.45454545454547), hastopping (54.54545454545454)
hastopping :  hastopping (100.0), hasbase (35.29411764705882)
hasingredient :  hasbase (54.0), hastopping (52.17391304347826)


In [34]:
common_obj_prop_pizza_text2owl=2

In [35]:
olaf_llm_obj_prop_uri = get_sparql_q_label_res(sparql_q=owl_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_obj_prop_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_llm_obj_prop_uri})

for true_label in pizza_onto_obj_prop_uri:
    res = process.extract(true_label, olaf_llm_obj_prop_uri, scorer=fuzz.WRatio, limit=3)
    print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

hasbase :  hasbase (100.0), hasbasil (80.0), haspizzabase (75.00000000000001)
isbaseof :  is (90.0), hasbase (66.66666666666667), flattenedbase (60.00000000000001)
hascountryoforigin :  hasorigin (72.0), hasdough (64.28571428571429), hasonion (60.00000000000001)
isingredientof :  is (90.0), hasingredient (81.4814814814815), haspizzaingredient (68.75)
istoppingof :  is (90.0), hastopping (76.19047619047619), hasvegetabletopping (70.00000000000001)
hasspiciness :  hasspiciness (100.0), haspizzaspiciness (82.75862068965517), hasbase (65.45454545454547)
hastopping :  hastopping (100.0), hasmeattopping (83.33333333333334), haspizzatopping (80.0)
hasingredient :  hasingredient (100.0), haspizzaingredient (83.87096774193549), hasonion (69.23076923076923)


In [36]:
common_obj_prop_pizza_olaf_llm = 5

In [37]:
olaf_no_llm_obj_prop_uri = get_sparql_q_label_res(sparql_q=owl_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_obj_prop_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_no_llm_obj_prop_uri})

for true_label in pizza_onto_obj_prop_uri:
    res = process.extract(true_label, olaf_no_llm_obj_prop_uri, scorer=fuzz.WRatio, limit=3)
    print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

hasbase :  based (66.66666666666667), are (60.00000000000001), used (60.00000000000001)
isbaseof :  based (72.0), are (60.00000000000001), baked (54.0)
hascountryoforigin :  ranging (60.00000000000001), consisting (45.0), prefer (45.0)
isingredientof :  are (60.00000000000001), fired (54.0), topping (45.0)
istoppingof :  topping (90.0), topped (60.00000000000001), consisting (57.14285714285714)
hasspiciness :  includes (55.38461538461539), suggests (49.090909090909086), include (45.0)
hastopping :  topping (82.35294117647058), topped (60.00000000000001), consisting (50.0)
hasingredient :  are (60.00000000000001), ranging (55.38461538461539), fired (54.0)


In [38]:
common_obj_prop_pizza_olaf_no_llm = 3

SubClassOf tuples

In [39]:
pizza_onto_sub_uri = get_sparql_q_tuple_res(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_sub_uri = list({f"{label[0].lower().replace(str(pizza_onto_ns), '')}#{label[1].lower().replace(str(pizza_onto_ns), '')}" for label in pizza_onto_sub_uri if not(label[0][0:2]=="nc" or label[1][0:2]=="nc")})

In [40]:
llm_text2owl_sub_uri = get_sparql_q_tuple_res(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_sub_uri = list({f"{label[0].lower().replace(str(olaf_eswc_ns), '')}#{label[1].lower().replace(str(olaf_eswc_ns), '')}" for label in llm_text2owl_sub_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_sub_uri:
#     res = process.extract(true_label, llm_text2owl_sub_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [41]:
common_sub_pizza_text2owl=17

In [42]:
olaf_llm_sub_uri = get_sparql_q_tuple_res(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_sub_uri = list({f"{label[0].lower().replace(str(olaf_eswc_ns), '')}#{label[1].lower().replace(str(olaf_eswc_ns), '')}" for label in olaf_llm_sub_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_sub_uri:
#     res = process.extract(true_label, olaf_llm_sub_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [43]:
common_sub_pizza_olaf_llm=27

In [44]:
olaf_no_llm_sub_uri = get_sparql_q_tuple_res(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_sub_uri = list({f"{label[0].lower().replace(str(olaf_eswc_ns), '')}#{label[1].lower().replace(str(olaf_eswc_ns), '')}" for label in olaf_no_llm_sub_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_sub_uri:
#     res = process.extract(true_label, olaf_no_llm_sub_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [45]:
common_sub_pizza_olaf_no_llm=6

Results

In [48]:
classes_precision_text2owl = common_classes_pizza_text2owl/len(llm_text2owl_class_labels)
classes_precision_olafLLM =  common_classes_pizza_olaf_llm/len(olaf_llm_class_labels)
classes_precision_olafNoLLM = common_classes_pizza_olaf_no_llm/len(olaf_no_llm_class_labels)
  
classes_recall_text2owl = common_classes_pizza_text2owl/len(pizza_onto_class_labels)
classes_recall_olafLLM = common_classes_pizza_olaf_llm/len(pizza_onto_class_labels)
classes_recall_olafNoLLM = common_classes_pizza_olaf_no_llm/len(pizza_onto_class_labels)

individuals_precision_text2owl = common_ind_pizza_text2owl/len(llm_text2owl_ind_uri)
individuals_precision_olafLLM =  common_ind_pizza_olaf_llm/len(olaf_llm_ind_uri)
individuals_precision_olafNoLLM = common_ind_pizza_olaf_no_llm/len(olaf_no_llm_ind_uri)

individuals_recall_text2owl = common_ind_pizza_text2owl/len(pizza_onto_ind_uri)
individuals_recall_olafLLM = common_ind_pizza_olaf_llm/len(pizza_onto_ind_uri)
individuals_recall_olafNoLLM = common_ind_pizza_olaf_no_llm/len(pizza_onto_ind_uri)

classesAndIndividuals_precision_text2owl = common_classes_ind_pizza_text2owl/len(llm_text2owl_classes_ind_uri)
classesAndIndividuals_precision_olafLLM = common_classes_ind_pizza_olaf_llm/len(olaf_llm_classes_ind_uri)
classesAndIndividuals_precision_olafNoLLM = common_classes_ind_pizza_olaf_no_llm/len(olaf_no_llm_classes_ind_uri)

classesAndIndividuals_recall_text2owl = common_classes_ind_pizza_text2owl/len(pizza_onto_classes_ind_uri)
classesAndIndividuals_recall_olafLLM = common_classes_ind_pizza_olaf_llm/len(pizza_onto_classes_ind_uri)
classesAndIndividuals_recall_olafNoLLM = common_classes_ind_pizza_olaf_no_llm/len(pizza_onto_classes_ind_uri)

objectProperties_precision_text2owl = common_obj_prop_pizza_text2owl/len(pizza_onto_obj_prop_uri)
objectProperties_precision_olafLLM = common_obj_prop_pizza_olaf_llm/len(olaf_llm_obj_prop_uri)
objectProperties_precision_olafNoLLM = common_obj_prop_pizza_olaf_no_llm/len(olaf_no_llm_obj_prop_uri)

objectProperties_recall_text2owl = common_obj_prop_pizza_text2owl/len(llm_text2owl_obj_prop_uri)
objectProperties_recall_olafLLM = common_obj_prop_pizza_olaf_llm/len(pizza_onto_obj_prop_uri)
objectProperties_recall_olafNoLLM = common_obj_prop_pizza_olaf_no_llm/len(pizza_onto_obj_prop_uri)

subClassOfPairs_precision_text2owl = common_sub_pizza_text2owl/len(llm_text2owl_sub_uri)
subClassOfPairs_precision_olafLLM = common_sub_pizza_olaf_llm/len(olaf_llm_sub_uri)
subClassOfPairs_precision_olafNoLLM = common_sub_pizza_olaf_no_llm/len(olaf_no_llm_sub_uri)

subClassOfPairs_recall_text2owl = common_sub_pizza_text2owl/len(pizza_onto_sub_uri)
subClassOfPairs_recall_olafLLM = common_sub_pizza_olaf_llm/len(pizza_onto_sub_uri)
subClassOfPairs_recall_olafNoLLM = common_sub_pizza_olaf_no_llm/len(pizza_onto_sub_uri)

In [49]:
# onto_evaluation = {
#     "Metrics": [
#         "Classes precision", 
#         "Classes recall", 
#         "Individuals precision", 
#         "Individuals recall", 
#         "Classes and individuals precision", 
#         "Classes and individuals recall", 
#         "Object properties precision", 
#         "Object properties recall", 
#         "SubClass of pairs precision", 
#         "SubClass of pairs recall"
#     ],
#     "Text to OWL": [
#         common_classes_pizza_text2owl/len(llm_text2owl_class_labels), 
#         common_classes_pizza_text2owl/len(pizza_onto_class_labels), 
#         common_ind_pizza_text2owl/len(llm_text2owl_ind_uri),
#         common_ind_pizza_text2owl/len(pizza_onto_ind_uri), 
#         common_classes_ind_pizza_text2owl/len(llm_text2owl_classes_ind_uri), 
#         common_classes_ind_pizza_text2owl/len(pizza_onto_classes_ind_uri), 
#         common_obj_prop_pizza_text2owl/len(pizza_onto_obj_prop_uri), 
#         common_obj_prop_pizza_text2owl/len(llm_text2owl_obj_prop_uri), 
#         common_sub_pizza_text2owl/len(llm_text2owl_sub_uri), 
#         common_sub_pizza_text2owl/len(pizza_onto_sub_uri)
#     ],
#     "OLAF LLM": [
#         common_classes_pizza_olaf_llm/len(olaf_llm_class_labels), 
#         common_classes_pizza_olaf_llm/len(pizza_onto_class_labels), 
#         common_ind_pizza_olaf_llm/len(olaf_llm_ind_uri), 
#         common_ind_pizza_olaf_llm/len(pizza_onto_ind_uri), 
#         common_classes_ind_pizza_olaf_llm/len(olaf_llm_classes_ind_uri), 
#         common_classes_ind_pizza_olaf_llm/len(pizza_onto_classes_ind_uri), 
#         common_obj_prop_pizza_olaf_llm/len(olaf_llm_obj_prop_uri), 
#         common_obj_prop_pizza_olaf_llm/len(pizza_onto_obj_prop_uri), 
#         common_sub_pizza_olaf_llm/len(olaf_llm_sub_uri),
#         common_sub_pizza_olaf_llm/len(pizza_onto_sub_uri)
#     ],
#     "OLAF no LLM": [
#         common_classes_pizza_olaf_no_llm/len(olaf_no_llm_class_labels),
#         common_classes_pizza_olaf_no_llm/len(pizza_onto_class_labels),
#         common_ind_pizza_olaf_no_llm/len(olaf_no_llm_ind_uri),
#         common_ind_pizza_olaf_no_llm/len(pizza_onto_ind_uri),
#         common_classes_ind_pizza_olaf_no_llm/len(olaf_no_llm_classes_ind_uri),
#         common_classes_ind_pizza_olaf_no_llm/len(pizza_onto_classes_ind_uri),
#         common_obj_prop_pizza_olaf_no_llm/len(olaf_no_llm_obj_prop_uri),
#         common_obj_prop_pizza_olaf_no_llm/len(pizza_onto_obj_prop_uri),
#         common_sub_pizza_olaf_no_llm/len(olaf_no_llm_sub_uri),
#         common_sub_pizza_olaf_no_llm/len(pizza_onto_sub_uri)
#     ]
# }

onto_evaluation = {
    "Metrics": [
        "Classes precision", 
        "Classes recall", 
        "Individuals precision", 
        "Individuals recall", 
        "Classes and individuals precision", 
        "Classes and individuals recall", 
        "Object properties precision", 
        "Object properties recall", 
        "SubClass of pairs precision", 
        "SubClass of pairs recall"
    ],
    "Text to OWL": [
        classes_precision_text2owl, 
        classes_recall_text2owl, 
        individuals_precision_text2owl,
        individuals_recall_text2owl, 
        classesAndIndividuals_precision_text2owl, 
        classesAndIndividuals_recall_text2owl, 
        objectProperties_precision_text2owl, 
        objectProperties_recall_text2owl, 
        subClassOfPairs_precision_text2owl, 
        subClassOfPairs_recall_text2owl
    ],
    "OLAF LLM": [
        classes_precision_olafLLM, 
        classes_recall_olafLLM, 
        individuals_precision_olafLLM, 
        individuals_recall_olafLLM, 
        classesAndIndividuals_precision_olafLLM, 
        classesAndIndividuals_recall_olafLLM, 
        objectProperties_precision_olafLLM, 
        objectProperties_recall_olafLLM, 
        subClassOfPairs_precision_olafLLM,
        subClassOfPairs_recall_olafLLM
    ],
    "OLAF no LLM": [
        classes_precision_olafNoLLM,
        classes_recall_olafNoLLM,
        individuals_precision_olafNoLLM,
        individuals_recall_olafNoLLM,
        classesAndIndividuals_precision_olafNoLLM,
        classesAndIndividuals_recall_olafNoLLM,
        objectProperties_precision_olafNoLLM,
        objectProperties_recall_olafNoLLM,
        subClassOfPairs_precision_olafNoLLM,
        subClassOfPairs_recall_olafNoLLM
    ]
}

df_evaluation = pd.DataFrame(onto_evaluation)

In [50]:
df_evaluation

,Metrics,Text to OWL,OLAF LLM,OLAF no LLM
0,Classes precision,1.000000,0.567010,0.387387
1,Classes recall,0.378947,0.578947,0.452632
2,Individuals precision,0.000000,0.010526,0.005848
3,Individuals recall,0.000000,0.200000,0.400000
4,Classes and individuals precision,0.539683,0.556701,0.130243
5,Classes and individuals recall,0.239437,0.380282,0.415493
6,Object properties precision,0.250000,0.064935,0.136364
7,Object properties recall,1.000000,0.625000,0.375000
8,SubClass of pairs precision,0.515152,0.236842,0.011583
9,SubClass of pairs recall,0.066148,0.105058,0.023346


In [54]:
def f1_score(precision, recall) -> float:
    if precision + recall == 0:
        return 0
    else:
        return (precision * recall) / (precision + recall)

In [55]:
onto_evaluation_f1_score = {
    "Metrics": [
        "Classes F1-score", 
        "Individuals F1-score", 
        "Classes and individuals F1-score", 
        "Object properties F1-score", 
        "SubClass of pairs F1-score" 
    ],
    "Text to OWL": [
        f1_score(classes_precision_text2owl, classes_recall_text2owl),
        f1_score(individuals_precision_text2owl, individuals_recall_text2owl),
        f1_score(classesAndIndividuals_precision_text2owl, classesAndIndividuals_recall_text2owl),
        f1_score(objectProperties_precision_text2owl, objectProperties_recall_text2owl),
        f1_score(subClassOfPairs_precision_text2owl, subClassOfPairs_recall_text2owl)
    ],
    "OLAF LLM": [
        f1_score(classes_precision_olafLLM, classes_recall_olafLLM),
        f1_score(individuals_precision_olafLLM, individuals_recall_olafLLM),
        f1_score(classesAndIndividuals_precision_olafLLM, classesAndIndividuals_recall_olafLLM),
        f1_score(objectProperties_precision_olafLLM, objectProperties_recall_olafLLM),
        f1_score(subClassOfPairs_precision_olafLLM, subClassOfPairs_recall_olafLLM)
    ],
    "OLAF no LLM": [
        f1_score(classes_precision_olafNoLLM, classes_recall_olafNoLLM),
        f1_score(individuals_precision_olafNoLLM, individuals_recall_olafNoLLM),
        f1_score(classesAndIndividuals_precision_olafNoLLM, classesAndIndividuals_recall_olafNoLLM),
        f1_score(objectProperties_precision_olafNoLLM, objectProperties_recall_olafNoLLM),
        f1_score(subClassOfPairs_precision_olafNoLLM, subClassOfPairs_recall_olafNoLLM),
    ]
}

df_evaluation_f1_score = pd.DataFrame(onto_evaluation_f1_score)

df_evaluation_f1_score

,Metrics,Text to OWL,OLAF LLM,OLAF no LLM
0,Classes F1-score,0.274809,0.286458,0.208738
1,Individuals F1-score,0.000000,0.010000,0.005764
2,Classes and individuals F1-score,0.165854,0.225941,0.099160
3,Object properties F1-score,0.200000,0.058824,0.100000
4,SubClass of pairs F1-score,0.058621,0.072776,0.007742


Text to OWL : 

- precision au top 30 concept extrait et tous bons
- mais extrait que 30 concepts sur les 97 de la pizza onto
- pas d'individu ? ou pas d'individus pertinents ?
- Beaucoup plus de propriété que dans la pizza onto mais du coup beaucoup de bruits.
- Il ne semble pas très bon sur la hierearchisation, peut de realtions hierarchiques qui sont précise mais du coup mauvais rapelle.


Remarque générale sur les LLMs :

Sans prompt spécifiques un GPT a tendance à construire des classes

Les LLMs c'est plus performant quand on les focus sur des tâches précises.